In [2]:
# 本代码由可视化策略环境自动生成 2021年12月10日 19:08
# 本代码单元只能在可视化模式下编辑。您也可以拷贝代码，粘贴到新建的代码单元或者策略，然后修改。


# Python 代码入口函数，input_1/2/3 对应三个输入端，data_1/2/3 对应三个输出端
def m5_run_bigquant_run(input_1, start_date,end_date):
    # 示例代码如下。在这里编写您的代码
    ins=input_1.read_pickle()['instruments']
    start_date=input_1.read_pickle()['start_date']
    end_date=input_1.read_pickle()['end_date']
    df=DataSource('bar1d_CN_FUND').read(start_date=start_date,end_date=end_date)
    #df = df[['date', 'close', 'instrument']].rename(columns={'close': 'close_0'})
    data_1 = DataSource.write_df(df)
    return Outputs(data_1=data_1, data_2=None, data_3=None)

# 后处理函数，可选。输入是主函数的输出，可以在这里对数据做处理，或者返回更友好的outputs数据格式。此函数输出不会被缓存。
def m5_post_run_bigquant_run(outputs):
    return outputs

# 回测引擎：初始化函数，只执行一次
def m3_initialize_bigquant_run(context):

    # 系统已经设置了默认的交易手续费和滑点，要修改手续费可使用如下函数
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

# 回测引擎：每日数据处理函数，每天执行一次
def m3_handle_data_bigquant_run(context, data):
    # 获取今日的日期
    today = data.current_dt.strftime('%Y-%m-%d')  
    # 通过positions对象，使用列表生成式的方法获取目前持仓的股票列表
    stock_hold_now = {e.symbol: p.amount * p.last_sale_price
                 for e, p in context.perf_tracker.position_tracker.positions.items()}

    # 记录用于买入股票的可用现金,因为是早盘卖股票，需要记录卖出的股票市值并在买入下单前更新可用现金；
    # 如果是早盘买尾盘卖，则卖出时不需更新可用现金，因为尾盘卖出股票所得现金无法使用
    cash_for_buy = context.portfolio.cash    
    
    try:
        buy_stock = context.daily_stock_buy[today]  # 当日符合买入条件的股票
    except:
        buy_stock=[]  # 如果没有符合条件的股票，就设置为空
    
    try:
        sell_stock = context.daily_stock_sell[today]  # 当日符合卖出条件的股票
    except:
        sell_stock=[] # 如果没有符合条件的股票，就设置为空
    
    # 需要卖出的股票:已有持仓中符合卖出条件的股票
    stock_to_sell = [ i for i in stock_hold_now if i in sell_stock ]
    # 需要买入的股票:没有持仓且符合买入条件的股票
    stock_to_buy = [ i for i in buy_stock if i not in stock_hold_now ]  
    # 需要调仓的股票：已有持仓且不符合卖出条件的股票
    stock_to_adjust=[ i for i in stock_hold_now if i not in sell_stock ]
    
    # 如果有卖出信号
    if len(stock_to_sell)>0:
        for instrument in stock_to_sell:
            sid = context.symbol(instrument) # 将标的转化为equity格式
            cur_position = context.portfolio.positions[sid].amount # 持仓
            if cur_position > 0 and data.can_trade(sid):
                context.order_target_percent(sid, 0) # 全部卖出 
                # 因为设置的是早盘卖出早盘买入，需要根据卖出的股票更新可用现金；如果设置尾盘卖出早盘买入，则不需更新可用现金(可以删除下面的语句)
                cash_for_buy += stock_hold_now[instrument]
    
    # 如果有买入信号/有持仓
    if len(stock_to_buy)+len(stock_to_adjust)>0:
        weight = 1/(len(stock_to_buy)+len(stock_to_adjust)) # 每只股票的比重为等资金比例持有
        for instrument in stock_to_buy+stock_to_adjust:
            sid = context.symbol(instrument) # 将标的转化为equity格式
            if  data.can_trade(sid):
                context.order_target_value(sid, weight*cash_for_buy) # 买入
# 回测引擎：准备数据，只执行一次
def m3_prepare_bigquant_run(context):
    # 加载预测数据
    df = context.options['data'].read_df()

    # 函数：求满足开仓条件的股票列表
    def open_pos_con(df):
        return list(df[df['buy_condition']>0].instrument)

    # 函数：求满足平仓条件的股票列表
    def close_pos_con(df):
        return list(df[df['sell_condition']>0].instrument)

    # 每日买入股票的数据框
    context.daily_stock_buy= df.groupby('date').apply(open_pos_con)
    # 每日卖出股票的数据框
    context.daily_stock_sell= df.groupby('date').apply(close_pos_con)

m1 = M.input_features.v1(
    features="""# #号开始的表示注释
# 多个特征，每行一个，可以包含基础特征和衍生特征
buy_condition=where(mean(close,5)>mean(close,10),1,0)
sell_condition=where(mean(close,5)<mean(close,10),1,0)""",
    m_cached=False
)

m2 = M.instruments.v2(
    start_date=T.live_run_param('trading_date', '2016-01-01'),
    end_date=T.live_run_param('trading_date', '2019-01-01'),
    market='CN_FUND',
    instrument_list='',
    max_count=0
)

m5 = M.cached.v3(
    input_1=m2.data,
    run=m5_run_bigquant_run,
    post_run=m5_post_run_bigquant_run,
    input_ports='',
    params='{}',
    output_ports=''
)

m8 = M.derived_feature_extractor.v3(
    input_data=m5.data_1,
    features=m1.data,
    date_col='date',
    instrument_col='instrument',
    drop_na=False,
    remove_extra_columns=False
)

m4 = M.dropnan.v2(
    input_data=m8.data
)

m3 = M.trade.v4(
    instruments=m2.data,
    options_data=m4.data,
    start_date='',
    end_date='',
    initialize=m3_initialize_bigquant_run,
    handle_data=m3_handle_data_bigquant_run,
    prepare=m3_prepare_bigquant_run,
    volume_limit=0.025,
    order_price_field_buy='open',
    order_price_field_sell='open',
    capital_base=1000000,
    auto_cancel_non_tradable_orders=True,
    data_frequency='daily',
    price_type='后复权',
    product_type='股票',
    plot_charts=True,
    backtest_only=False,
    benchmark=''
)


[2020-11-10 16:07:55.571304] INFO: moduleinvoker: input_features.v1 开始运行..

[2020-11-10 16:07:55.614960] INFO: moduleinvoker: input_features.v1 运行完成[0.043659s].

[2020-11-10 16:07:55.621873] INFO: moduleinvoker: instruments.v2 开始运行..

[2020-11-10 16:07:55.631850] INFO: moduleinvoker: 命中缓存

[2020-11-10 16:07:55.633366] INFO: moduleinvoker: instruments.v2 运行完成[0.011494s].

...

[2020-11-10 16:07:59.282125] INFO: moduleinvoker: derived_feature_extractor.v3 运行完成[3.272891s].

[2020-11-10 16:07:59.288206] INFO: moduleinvoker: dropnan.v2 开始运行..

[2020-11-10 16:07:59.385451] INFO: dropnan: /data, 3452/3452

[2020-11-10 16:07:59.779698] INFO: dropnan: 行数: 3452/3452

[2020-11-10 16:07:59.786408] INFO: moduleinvoker: dropnan.v2 运行完成[0.498193s].